# OpenStreetMap
## Downloading data from OpenStreetMap
*Written by - Rasmus Bergman rbvp20@student.aau.dk*

This is a notebook for downloading the data needed from OpenStreetMap.

It downloads and saves a graph of the entire road network of Denmark. This may take a while.

In [3]:
# Import libraries
import os
import osmnx as ox
import constants as c
import geopandas as gpd
# Config
ox.settings.console_log = True
ox.settings.use_cache = True

# Parameters
EDGE_CHUNK_SIZE = 100_000

In [ ]:
edges = gpd.read_file(c.EDGE_GDF_PATH)

# Split up the edges into smaller chunks

# Get the number of chunks
num_chunks = len(edges) // EDGE_CHUNK_SIZE + 1

# Split the edges into chunks
for i in range(num_chunks):
    chunk = edges.iloc[i * EDGE_CHUNK_SIZE: (i + 1) * EDGE_CHUNK_SIZE]
    chunk.to_file(c.EDGE_GDF_PATH.replace('.geojson', f'_{i}.geojson'), driver='GeoJSON')
    print(f'Chunk {i} saved')

In [6]:
# Downloading and saving the road network

print("\nLoading simplified road network")
if not os.path.exists(c.SIMPLE_ROAD_NETWORK_PATH):
    print("- Downloading simplified road network")
    G = ox.graph_from_place('Denmark', network_type='drive', simplify=True)
    print("- Saving")
    ox.save_graphml(G, c.SIMPLE_ROAD_NETWORK_PATH)
    print("- Saved")
else:
    print("- Already exists")
G = None

print("\nLoading full road network")
if not os.path.exists(c.FULL_ROAD_NETWORK_PATH):
    print("- Downloading full road network, this may take quite a while (eg. 30 - 60 minutes)")
    G = ox.graph_from_place('Denmark', network_type='drive', simplify=False)
    print("- Saving")
    ox.save_graphml(G, c.FULL_ROAD_NETWORK_PATH)
    print("- Saved")
else:
    print("- Already exists")


print("\nLoading node GeoDataFrame")
if not os.path.exists(c.NODE_GDF_PATH):
    if G is None:
        print("- Loading full road network")
        G = ox.load_graphml(c.FULL_ROAD_NETWORK_PATH)
    print("- Transforming to GeoDataFrame")
    node_gdf = ox.graph_to_gdfs(G, edges=False)
    print("- Saving")
    node_gdf.to_file(c.NODE_GDF_PATH, driver='GeoJSON')
    print("- Saved")
else:
    print("- Already exists")

print("\nLoading edge GeoDataFrame")
if not os.path.exists(c.EDGE_GDF_PATH):
    if G is None:
        print("- Loading full road network")
        G = ox.load_graphml(c.FULL_ROAD_NETWORK_PATH)
    print("- Transforming to GeoDataFrame")
    edge_gdf = ox.graph_to_gdfs(G, nodes=False)
    print("- Saving")
    edge_gdf.to_file(c.EDGE_GDF_PATH, driver='GeoJSON')
    print("- Saved")
else:
    print("- Already exists")



Loading simplified road network
- Already exists

Loading full road network
- Already exists

Loading node GeoDataFrame
- Already exists

Loading edge GeoDataFrame
- Already exists
